# Dunham's Data

## Setting up

In [101]:
from geopy.distance import geodesic
# import networkx as nx
import pandas as pd
# import seaborn as sns

In [102]:
# pd.set_option('display.max_rows', 999)

In [103]:
# sns.set()

In [104]:
# %matplotlib inline

## Loading data

In [105]:
col_used = [
    'DATE', 'CITY1', 'COUNTRY1', 'CITY2', 'COUNTRY2', 'WORKING', 'HOTEL_ADDRESS',
    'VENUE1', 'VENUE_TYPE1', 'VENUE2', 'VENUE_TYPE2', 'NOTES', 'SOURCE',
]
dunham_df = pd.read_excel('data/src/nureyev_EverydayItinerary_DatasetSample.xlsx',
                          usecols='A:G,I:N', names=col_used)
dunham_df.fillna('', inplace=True)

In [106]:
dunham_df.shape

(22, 13)

In [107]:
dunham_df.head()

,DATE,CITY1,COUNTRY1,CITY2,COUNTRY2,WORKING,HOTEL_ADDRESS,VENUE1,VENUE_TYPE1,VENUE2,VENUE_TYPE2,NOTES,SOURCE
0,1975-09-01,Edinburgh,Scotland,,,y,,Edinburgh festival,,,,,
1,1975-09-07,Glasgow,Scotland,,,y,,Reharse scottish ballet glasgow,,,,,
2,1975-09-16,Madrid,Spain,,,y,,,,,,,
3,1975-09-22,Amsterdam,Holland,,,y,,Reharse dutch ballet amsterdam,,,,,
4,1975-10-17,Vienna,Austria,,,y,,,,,,,


In [108]:
dunham_df.tail(1)

,DATE,CITY1,COUNTRY1,CITY2,COUNTRY2,WORKING,HOTEL_ADDRESS,VENUE1,VENUE_TYPE1,VENUE2,VENUE_TYPE2,NOTES,SOURCE
21,1976-02-15,Los Angeles,California,,,y,,With american ballet theatre,,,,,


## Pre-processing data

In [109]:
dunham_df.DATE = dunham_df.DATE.astype(str)
for col in dunham_df.columns[1:]:
    dunham_df[col] = dunham_df[col].apply(str.strip)

In [110]:
def clean_city(x, y):
    if x == 'Hamilton':
        if y == 'Canada':
            r = 'Hamilton (CA)'
        else: # y == 'New Zealand'
            r = 'Hamilton (NZ)'
#     elif x == 'in transit':
#         r = ''
    else:
        r = x
    return r

dunham_df.CITY1 = dunham_df.apply(lambda row: clean_city(row.CITY1, row.COUNTRY1), axis=1)
dunham_df.CITY2 = dunham_df.apply(lambda row: clean_city(row.CITY2, row.COUNTRY2), axis=1)

In [111]:
def clean_country(city, country):
    x = country
    y = city
    if y == 'Gibraltar':
        r = 'Gibraltar'
    elif x == 'Newfoundland':
        r = 'Canada'
    else:
        r = x
    return r

dunham_df.COUNTRY1 = dunham_df.apply(lambda row: clean_country(row.CITY1, row.COUNTRY1), axis=1)
dunham_df.COUNTRY2 = dunham_df.apply(lambda row: clean_country(row.CITY2, row.COUNTRY2), axis=1)

In [112]:
# def clean_working(x):
#     if x.startswith('[[') and x.endswith(']]'):
#         r = ''
#     else:
#         r = x
#     return r

# dunham_df.WORKING = dunham_df.WORKING.apply(clean_working)

In [113]:
def clean_hotel_address(x):
    r = x # do not clean for now
    return r

dunham_df.HOTEL_ADDRESS = dunham_df.HOTEL_ADDRESS.apply(clean_hotel_address)

In [114]:
# def clean_performance_yn(x):
#     if x == '?':
#         r = ''
#     elif x == 'N':
#         r = 'n'
#     else:
#         r = x
#     return r

# dunham_df.PERFORMANCE_YN = dunham_df.PERFORMANCE_YN.apply(clean_performance_yn)

In [115]:
# def clean_theater1(x):
#     r = x # do not clean for now
#     return r

# dunham_df.THEATER1 = dunham_df.THEATER1.apply(clean_theater1)

In [116]:
# def clean_theater2(x):
#     r = x # do not clean for now
#     return r

# dunham_df.THEATER2 = dunham_df.THEATER2.apply(clean_theater2)

In [117]:
# def clean_mode_transit(x):
#     if x == '[[chartered bus Reno to Los Angeles?]]':
#         r = 'bus'
#     elif x == 'drive':
#         r = 'car'
#     elif x == 'drive?':
#         r = 'car'
#     elif x.startswith('plane'):
#         r = 'plane'
#     elif x == 'train; boat train':
#         r = 'train, boat, train'
#     elif x == 'train; plane; boat':
#         r = 'train, plane, boat'
#     else:
#         r = x
#     return r

# dunham_df.MODE_TRANSIT = dunham_df.MODE_TRANSIT.apply(clean_mode_transit)

In [118]:
# def clean_transit_cities(x):
#     if x == 'Geneva; Rome':
#         r = 'Geneva (Switzerland), Rome (Italy)'
#     elif x == 'transit to Turin':
#         r = ''
#     elif x == 'transit through London':
#         r = 'London (England)'
#     elif x == 'transit: PanAm 201 flies NYC Laguardia - San Juan - Port of Spain - Belem':
#         r = 'San Juan (Puerto Rico), Port of Spain (Trinidad and Tobago), Belem (Brazil)'
#     elif x == 'Los Cerrillos airport':
#         r = ''
#     elif x == 'transit':
#         r = ''
#     elif x == 'destination':
#         r = ''
#     elif x == 'transit via Boston':
#         r = 'Boston (USA)'
#     elif x == 'in transit':
#         r = ''
#     elif x == 'last performance':
#         r = ''
#     elif x == 'London (Liverpool station), England; Harwich (Harwich Quay), England':
#         r = 'Harwich (England)'
#     elif x == 'New York':
#         r = ''
#     elif x == 'Chicago':
#         r = 'Chicago (USA)'
#     elif x == 'Gander, Newfoundland, Canada':
#         r = 'Gander (Canada)'
#     elif x == 'Zurich, Switzerland':
#         r = 'Zurich (Switzerland)'
#     elif x == 'Zurich':
#         r = 'Zurich (Switzerland)'
#     elif x == 'Rome, Italy; Geneva, Switzerland':
#         r = 'Rome (Italy), Geneva (Switzerland)'
#     elif x == 'Vevey, Switzerland':
#         r = 'Vevey (Switzerland)'
#     elif x == 'Amsterdam, Netherlands':
#         r = 'Amsterdam (Netherlands)'
#     elif x == 'Furet de P*?': # ???
#         r = ''
#     elif x == 'San Diego; Tijuana':
#         r = 'San Diego (USA), Tijuana (Mexico)'
#     elif x == 'Portworth':
#         r = 'Fort Worth (USA)'
#     elif x == 'Miami':
#         r = 'Miami (USA)'
#     elif x == 'Los Angeles':
#         r = 'Los Angeles (USA)'
#     elif x == "El Paso, TX----I think probably this was to go to Mexico, and I don't think she ever went to Mexico":
#         r = 'El Paso (USA)'
#     elif x == 'Honolulu':
#         r = 'Honolulu (Hawaii)'
#     elif x == 'Coolangatta':
#         r = 'Coolangatta (Australia)'
#     elif x == '[[Dunham Brisbane to Perth via Sydney and Melbourne--date open: https://osu.app.box.com/file/325139873147]]':
#         r = ''
#     elif x == 'Malacca':
#         r = 'Malacca (Malaysia)'
#     elif x == 'Amsterdam':
#         r = 'Amsterdam (Netherlands)'
#     elif x == 'Lyon; Bordeaux; Bayonne':
#         r = 'Lyons (France), Bordeaux (France), Bayonne (France)'
#     elif x == 'Irun, Spain':
#         r = 'Irun (Spain)'
#     elif x == 'Irun, Spain; Hendaye, France':
#         r = 'Irun (Spain), Hendaye (France)'
#     elif x == 'Marseilles':
#         r = 'Marseille (France)'
#     elif x == 'Piraeus, Greece':
#         r = 'Piraeus (Greece)'
#     elif x == 'Port Said, Egypt':
#         r = 'Port Said (Egypt)'
#     elif x == 'Munich':
#         r = 'Munich (Germany)'
#     elif x == 'Schwarzbach; Ravensberg':
#         r = 'Schwarzbach (Germany), Ravensberg (Germany)'
#     elif x == 'Colmar; Ligny en Barois; Rosay en Brie':
#         r = 'Colmar (France), Ligny-en-Barrois (France), Rozay-en-Brie (France)'
#     elif x == 'Verneson; Valence':
#         r = 'Vernaison (France), Valence (France)'
#     elif x == 'Alicante':
#         r = 'Alicante (Spain)'
#     elif x == 'Granada':
#         r = 'Granada (Spain)'
#     elif x == 'La Linea, Spain':
#         r = 'La Linea (Spain)'
#     else:
#         r = x
#     return r

# dunham_df.TRANSIT_CITIES = dunham_df.TRANSIT_CITIES.apply(clean_transit_cities)

In [119]:
def clean_notes(x):
    r = x # do not clean for now
    return r

dunham_df.NOTES = dunham_df.NOTES.apply(clean_notes)

In [120]:
def clean_source(x):
    r = x # do not clean for now
    return r

dunham_df.SOURCE = dunham_df.SOURCE.apply(clean_source)

In [121]:
dunham_df.fillna('', inplace=True)

## Processing data

In [122]:


geolocations = { # (lat, lon)
 'Edinburgh'        :(55.9533456,-3.1883749 ),
 'Glasgow'          :(55.861155, -4.2501687 ),
 'Madrid'           :(40.4167047,-3.7035825 ),
 'Amsterdam'        :(52.3730796,4.8924534 ),
 'Vienna'           :(48.2083537,16.3725042 ),
 'Zurich'           :(47.3744489,8.5410422 ),
 'Lugano'           :(46.0050102,8.9520281 ),
 'Lausanne'         :(46.5218269,6.6327025 ),
 'Basel'            :(47.5581077,7.5878261 ),
 'Bregenz'          :(47.5025779,9.7472924 ),
 'Gallen'           :(51.4312285,12.5495463 ),
 'Geneva'           :(46.2017559,6.1466014 ),
 'The Hague'        :(52.0799838,4.3113461 ),
 'New York'         :(40.7127281,-74.0060152 ),
 'Brooklin'         :(40.691392, -73.982469),
 'London'           :(51.5074456,-0.1277653 ),
 'Manchester'       :(53.4794892,-2.2451148 ),
 'London'           :(51.5074456,-0.1277653 ),
 'New York'         :(40.7127281,-74.0060152 ),
 'London'           :(51.5074456,-0.1277653 ),
 'Paris'            :(48.8588897,2.320041022 ),
 'Los Angeles'      :(34.0536909,-118.242766 ),
} 




### Grouping locations

In [123]:
distances = set()
cities = set(dunham_df.CITY1.unique()).union(dunham_df.CITY2.unique())
cities.discard('')
cities.discard('in transit')
for i, city1 in enumerate(cities):
    p1 = geolocations[city1]
    for j, city2 in enumerate(cities):
        if j > i: # and city1 != city2:
            #print(city1)
            #print(city2)
            p2 = geolocations[city2]
            dst = geodesic(p1, p2).km
            distances.add((city1, city2, dst))

In [124]:
len(distances)

171

In [125]:
sorted(distances, key=lambda x: x[2])[:10]

[('New York', 'Brooklin', 3.0941360093068644),
 ('Amsterdam', 'The Hague', 51.38368920750861),
 ('Lausanne', 'Geneva', 51.625911775686106),
 ('Glasgow', 'Edinburgh', 67.19474201927635),
 ('Zurich', 'Basel', 74.7086969554593),
 ('Bregenz', 'Zurich', 92.09663768774247),
 ('Basel', 'Lausanne', 136.16389530656033),
 ('Zurich', 'Lugano', 155.4450446566572),
 ('Bregenz', 'Basel', 162.72002783335407),
 ('Zurich', 'Lausanne', 173.4594195432046)]

In [126]:
THRESHOLD = 15

In [127]:
grouped_cities_dct = {
    'Petion-Ville': 'Port-au-Prince',
    'Los Angeles': 'Los Angeles area',
    'Hollywood': 'Los Angeles area',
    'Beverly Hills': 'Los Angeles area',
    'Fiesole': 'Florence',
#     'Mosman': 'Sydney',
}

In [128]:
for k in grouped_cities_dct:
    dunham_df.replace(k, grouped_cities_dct[k], inplace=True)

In [129]:
dunham_df.shape

(22, 13)

## Saving data

In [130]:
dunham_df.to_csv('data/out/nureyev_1955-92.csv', index=False)

PermissionError: [Errno 13] Permission denied: 'data/out/nureyev_1955-92.csv'